# LangChain Chat History Management
In the [LangChain Expression Language (LCEL)](./lcel.ipynb), we covered LCEL at a high level, demonstrating specifically how to chain a prompt engineered chat prompt with an LLM, namely MLX. What I failed to demonstrate in that notebook was how to think about memory (aka chat conversation management), and to be honest, I underestimated how "involved" of a thing this got to be! 😅 To be clear, what we will be covering in this notebook is less of a technical concern and more of a business logic concern.

While LangChain offers many mechanisms for handling chat conversations (aka memory) correctly, I found some of the higher level ones to not be satisfactory for our purposes. Specifically, since I want us to adhere to a fixed schema, the high level abstraction objects provided by LangChain simply don't operate in the ideal way in which we need them to. No worries! We can still work around this without having to abandon LangChain. We're just going to need to do some special stuff throughout this notebook!

## High Level Flow
Before we get into the code itself, let's talk about how we want to think about the flow. For simplicity's sake, we are going to be ultimately saving this chat history as a JSON file. This JSON file should look like the schema that we've defined in the file `data/schema.json`.

Let's say that the user is loading the MLX Gradio UI interface, either for the first time ever or as a returning user. Here is the flow of how we should be thinking about our data:

1. **Loading the chat history from file**: Just as it sounds, we will want to load the chat history from file so that the user can interact with their historical conversations if they would like. Now, it's possible that this is the user's first time interacting with the chatbot, so it may be that we need to create this file from scratch!
2. **Setting a new conversation ID**: Regardless if the user is new or returning, we are going to make the assumption that the user will want to begin with a new conversation. This means that we will need to instantiate a new conversation ID so that we can keep appending new conversation interactions to that same conversation thread.
3. **Managing conversation back-and-forth**: As the conversation proceeds, we will want to continually update our conversation schema with any new human and AI interactions. This will include also autosaving them to file for the user's convenience.
4. **Starting a new conversation / loading an existing conversation**: At any point, the user may want to pivot from their current conversation to either a new conversation or to continue another historical conversation loaded from our file as part of step 1. If this is the case, we will need to ensure that our backend system is referencing the correct conversation interaction.

To really drive home the point, we will actually jump back and forth between each of these use cases to ensure that everything works seamlessly!

## Notebook Setup
In this section, we'll do all our usual set ups. We'll also set up the LangChain MLX model using the new ChatMLX implementation. All these are things we've already explored in other notebooks.

In [1]:
# Importing the necessary Python libraries
import os
import json
import uuid
import copy
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain_community.llms.mlx_pipeline import MLXPipeline
from langchain_community.chat_models.mlx import ChatMLX
from langchain_core.runnables import RunnableLambda
from langchain_community.chat_message_histories.in_memory import ChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [2]:
# Setting constant values to represent model name and directory
MODEL_NAME = 'mistralai/Mistral-7B-Instruct-v0.2'
BASE_DIRECTORY = '../models'
MLX_DIRECTORY = f'{BASE_DIRECTORY}/mlx'
mlx_model_directory = f'{MLX_DIRECTORY}/{MODEL_NAME}'

# Setting a constant value to represent where to place the chat history data
CHAT_HISTORY_DIRECTORY = '../data'
chat_history_json_location = f'{CHAT_HISTORY_DIRECTORY}/chat_history.json'

In [3]:
# Setting a default system prompt
DEFAULT_SYSTEM_PROMPT = 'You are a helpful assistant.'

class MLXModelParameters():

    def __init__(self, model_name = MODEL_NAME, temp = 0.7, max_tokens = 1000, system_prompt = DEFAULT_SYSTEM_PROMPT):
        self.model_name = model_name
        self.temp = temp
        self.max_tokens = max_tokens
        self.system_prompt = system_prompt

    def __str__(self):
        return f'Model Name: {self.model_name}\nTemperature: {self.temp}\nMax Tokens: {self.max_tokens}\nSystem Prompt: {self.system_prompt}'
    
    def __repr__(self):
        return f'Model Name: {self.model_name}\nTemperature: {self.temp}\nMax Tokens: {self.max_tokens}\System Prompt: {self.system_prompt}'
    
    def update_model_name(self, new_model_name):
        self.model_name = new_model_name

    def update_temp(self, new_temp):
        self.temp = new_temp

    def update_max_tokens(self, new_max_tokens):
        self.max_tokens = new_max_tokens

    def update_system_prompt(self, new_system_prompt):
        self.system_prompt = new_system_prompt

    def to_json(self):
        return { 'temp': self.temp, 'max_tokens': self.max_tokens }
    
mlx_model_parameters = MLXModelParameters()

In [4]:
# Setting up the LangChain MLX LLM
llm = MLXPipeline.from_model_id(
    model_id = mlx_model_directory,
    pipeline_kwargs = {
        'temp': mlx_model_parameters.temp,
        'max_tokens': mlx_model_parameters.max_tokens,
    }
)

# Setting up the LangChain MLX Chat Model with the LLM above
chat_model = ChatMLX(llm = llm)

# NOTE: The commented out code below will overwrite the MLX chat model with OpenAI. I did this out of curiosity, because theoretically, this should have worked just fine. And it did! I'm pretty thrilled about it!
# import yaml
# from langchain_openai import ChatOpenAI

# with open('../sensitive/api-keys.yaml') as f:
#     API_KEYS = yaml.safe_load(f)

# chat_model = ChatOpenAI(api_key = API_KEYS['OPENAI_API_KEY'])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Setting up the LangChain inference pipeline
In order to use LangChain's preferred implementation of memory management, we're first going to need to establish our LangChain pipeline. We've done similar things to this in other notebooks, but with this particular implementation, we are going to make a specific adjustment. Namely, since we are now going to make use of the LangChain Community implementation of MLX, we are going to need to manually add our own metadata. To seamlessly do this, we are going to make use of LCEL's **RunnableLambda**, which essentially allows us to define our own custom function.

Also note that when we set up our chat prompt, we are going to need to slide in an extra entry referred to as **MessagesPlaceholder**. As the name implies, that will serve as a placeholder so that we can keep passing the history back through the model.

Another item of note: Specific models like Llama or Mistral **do not** natively accept system messages. As a result, we're going to have to use that same **RunnableLambda** object to create another custom function that manages that appropriate conversion for those models.

To put it all together, here are the steps of our inference pipeline:

1. **Chat prompt**: This is the prompt engineering structure that we'll be eventually passing into the model. As mentioned above, this will also use the `MessagesPlaceholder` object to manage the history.
2. **Correct for "No System" models**: Because model providers like Meta and Mistral do not natively accept system messsages, this step in the pipeline will use the LCEL's `RunnableLambda` to create a custom function taht corrects for this issue manually.
3. **Invoke the model with MLX**: Pretty self explanatory. This is the step where we pass in our messages and any history to produce a response from the model.
4. **Update the AI message metadata**: Because LangChain's MLX object does not place any metadata on the AI message coming out of the MLX model, we are going to have to use another `RunnableLambda` here with a custom function that will allow us to manually add that metadata>

In [5]:
# Setting up the Chat prompt template
chat_prompt_template = ChatPromptTemplate.from_messages(messages = [
    SystemMessage(content = mlx_model_parameters.system_prompt),
    MessagesPlaceholder(variable_name = 'history'),
    HumanMessagePromptTemplate.from_template(template = '{input}')
])

In [6]:
def correct_for_no_system_models(chat_messages):
    '''
    Precorrects for the issue where certain models (e.g. Llama, Mistral) are unable to accept for system messages

    Inputs:
        - chat_messages (LangChain ChatPromptValue): The current chat messages with no alterations

    Returns:
        - chat_messages (LangChain ChatPromptValue): The new chat messages with alterations (if needed)
    '''
    
    # Referencing MLX model parameters as a global object
    global mlx_model_parameters
    
    # Setting a list of models that we'll need to check against
    NO_SYSTEM_MODEL_PROVIDERS = ['mistralai', 'meta-llama']

    # Checking if the correction needs to be made if the model is Llama or Mistral
    if mlx_model_parameters.model_name.split('/')[0] in NO_SYSTEM_MODEL_PROVIDERS:

        # Getting the system message content
        system_message_content = chat_messages.messages[0].content

        # Replacing the System Message with a Human Message
        chat_messages.messages[0] = HumanMessage(content = system_message_content)

        # Adds a dummy AI Message
        chat_messages.messages.insert(1, AIMessage(content = ''))

    return chat_messages

In [7]:
def update_ai_response_metadata(ai_message):
    '''
    Updates the metadata on the AI response

    Inputs:
        - ai_message (LangChain AIMessage): The AI message produced by the model

    Returns:
        - ai_message (LangChain AIMessage): The AI message produced by the model, except now with the appropriate metadata intact
    '''

    # Referencing MLX model parameters as a global object
    global mlx_model_parameters

    # Creating a dictionary of the metadata that we will be adding to the AI message
    metadata = {
        'model_name': mlx_model_parameters.model_name,
        'timestamp': str(pd.Timestamp.utcnow()),
        'like_data': None,
        'hyperparameters': mlx_model_parameters.to_json()
    }

    # Applying the metadata to the AI response
    ai_message.response_metadata = metadata

    return ai_message

In [8]:
# Creating the inference chain by chaining together the chat prompt, chat model, and custom function to update metadata
inference_chain = chat_prompt_template | RunnableLambda(correct_for_no_system_models) | chat_model | RunnableLambda(update_ai_response_metadata)

In [9]:
# Generating the response with the first prompt
response = inference_chain.invoke({
    'history': [
        HumanMessage(content = 'What is the capital of Illinois?'),
        AIMessage(content = 'The capital of Illinois is Sprinfield.')
    ],
    'input': 'What is the largest city in that state?'
})
print(response.content)

The largest city in Illinois is Chicago. Chicago is the third most populous city in the United States and the most populous city in the Midwest. It is located in the northeastern part of Illinois and is known for its iconic skyline, major industries, cultural institutions, and its role as a global hub for commerce, transportation, and technology.


## Managing the Conversation History
Now that we have instantiated our inference pipeline, we are ready to talk about managing the conversation history. The way that LangChain recommends to do this is by using this object called `RunnableWithMessageHistory`. What this function does is take in our "runnable" inference pipeline, and anything that is invoked from that gets added to the chat history over time. The nice thing about this object is that at the time of inference, you can also pass in a unique session ID and manage chat histories by that unique session ID. We'll demonstrate how we can use this to easily pivot back and forth between various conversations!

In [10]:
# Instantiating an empty dictionary to represent the user's LangChain (lc) conversation history
lc_user_conversation_history = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    '''
    Gets the conversation session history per a uniquely input session ID

    Inputs:
        - session_id (str): A unique ID representing the current conversation session

    Returns:
        - lc_user_conversation_history[session_id] (LangChain BaseChatMessageHistory): A LangChain history object with the history of the conversation per the respective session ID
    '''
    if session_id not in lc_user_conversation_history:
        lc_user_conversation_history[session_id] = ChatMessageHistory()

    return lc_user_conversation_history[session_id]

In [11]:
# Creating a LangChain invokable that will run the inference pipeline and also manage chat history
inference_chain_w_history = RunnableWithMessageHistory(
    runnable = inference_chain,
    get_session_history = get_session_history,
    input_messages_key = 'input',
    history_messages_key = 'history'
)

In [12]:
# Creating 2 demo IDs
demo_id_1 = 'conv_id_' + str.replace(str(uuid.uuid4()), '-', '_')
demo_id_2 = 'conv_id_' + str.replace(str(uuid.uuid4()), '-', '_')

In [13]:
# Invoking the model twice while passing in the session ID to manage the history
inference_chain_w_history.invoke(
    input = {'input': 'What is the capital of Illinois?'},
    config = {
        'configurable': {
            'session_id': demo_id_1
        }
    }
)

inference_chain_w_history.invoke(
    input = {'input': 'What is the largest city in that state?'},
    config = {
        'configurable': {
            'session_id': demo_id_1
        }
    }
)

inference_chain_w_history.invoke(
    input = {'input': 'What is the capital of California?'},
    config = {
        'configurable': {
            'session_id': demo_id_2
        }
    }
)

inference_chain_w_history.invoke(
    input = {'input': 'What is the largest city in that state?'},
    config = {
        'configurable': {
            'session_id': demo_id_2
        }
    }
)

AIMessage(content='The largest city in California by population is Los Angeles. Los Angeles is located in the southern part of the state and is known for its iconic Hollywood film industry, diverse cultural scene, and world-renowned attractions such as the Hollywood Walk of Fame, Griffith Observatory, and Universal Studios Hollywood. According to the latest US Census data, the population of Los Angeles County, which includes the city of Los Angeles, is over 10 million people, making it the most populous county', response_metadata={'model_name': 'mistralai/Mistral-7B-Instruct-v0.2', 'timestamp': '2024-05-02 14:56:34.368788+00:00', 'like_data': None, 'hyperparameters': {'temp': 0.7, 'max_tokens': 1000}}, id='run-88214bde-5a26-40dd-a3c3-eecaf2d587ed-0')

In [14]:
# Viewing the current user conversation history
print(lc_user_conversation_history)
print('\n')
print(lc_user_conversation_history[demo_id_1])
print('\n')
print(lc_user_conversation_history[demo_id_2])

{'conv_id_4c49e021_4dc1_4b2f_9892_511e810fb7e3': ChatMessageHistory(messages=[HumanMessage(content='What is the capital of Illinois?'), AIMessage(content="The capital city of Illinois is Springfield. It's located in the central part of the state and is the county seat of Sangamon County. Springfield is known for its rich history, including being the site of Abraham Lincoln's presidential library and his former home, which is now a National Historic Site.", response_metadata={'model_name': 'mistralai/Mistral-7B-Instruct-v0.2', 'timestamp': '2024-05-02 14:56:24.750245+00:00', 'like_data': None, 'hyperparameters': {'temp': 0.7, 'max_tokens': 1000}}, id='run-73e2b3b4-e799-4e0c-8234-a91fb956bb50-0'), HumanMessage(content='What is the largest city in that state?'), AIMessage(content="The largest city in Illinois is Chicago. Chicago is located in the northeastern part of the state and is the third most populous city in the United States. It's known for its iconic skyline, major industries, cu

## (Optional) Generating a Summary Title
In certain interfaces including OpenAI's ChatGPT, the chat history will represent your conversation with what I like to call a "summary title." For example, let's say I ask it for a recipe for chocolate chip cookies, then the ChatGPT interface will represent my conversation in the chat history window as something like "A Recipe for Chocolate Chip Cookies". While this is not necessary, I thought it might be a fun touch to add!

Let's demonstrate by starting a conversation asking it to write a fun haiku.

In [15]:
def generate_summary_title(lc_current_session_history, chat_model):
    '''
    Generates a summary title based on the user's initial interaction with the MLX model

    Inputs:
        - lc_current_session_history (LangChain ChatMessageHistory): A list of messages representing the current session conversation history
        - chat_model (LangChain MLX ChatModel): The chat model used to generate the summary title

    Returns:
        - summary_title (str): The summary title from the user's initial interaction with the MLX model
    '''

    # Referencing MLX model parameters as a global object
    global mlx_model_parameters

    # Creating the summary title prompt engineering
    summary_title_prompt = '''The text delineated by the triple backticks below contains the beginning of a conversation between a human and a large language model (LLM). Please provide a brief summary to serve as a title for this conversation. Do not use any system messages. Place more emphasis on the human's prompt over the AI's response. Please ensure the summary title does not exceed more than ten words. Please format the summary title as one would any formal title, like that of a book. Do not give any extra words except the summary title. (Example: Do not show "Title: ") Please ensure that the length of the output is no longer than 10 words.

    ```
    {history}
    ```
    '''

    # Creating the summary title chat prompt
    summary_title_chat_prompt = ChatPromptTemplate.from_messages(messages = [
        HumanMessagePromptTemplate.from_template(template = summary_title_prompt)
    ])

    # Creating a simple chain to produce the summary title
    summary_title_chain = summary_title_chat_prompt | chat_model
    
    # Getting just the first interaction from the chat message history
    initial_interaction = lc_current_session_history.messages[:2]

    # Generating the summary title based on the sample chat history
    summary_title_response = summary_title_chain.invoke({
        'history': initial_interaction
    })

    # Stripping out "Title: " (Because no idea why it wants to keep doing that...)
    summary_title = summary_title_response.content.replace("Title: ", "").replace("title: ", "")

    return summary_title

In [16]:
# Producing a summary title based on our demo sessions
summary_title_1 = generate_summary_title(lc_current_session_history = lc_user_conversation_history[demo_id_1], chat_model = chat_model)
print(summary_title_1)
summary_title_2 = generate_summary_title(lc_current_session_history = lc_user_conversation_history[demo_id_2], chat_model = chat_model)
print(summary_title_2)

Capital City of Illinois: Springfield
Capital City of California: Sacramento


## Starting the User History Schema
As mentioned before, we are going to be emulating the structure of the schema as defined in `data/schema.json`. In this notebook, we are going to pretend as if the user is a brand new user, so we will need to set up the conversation history schema from scratch.

In [17]:
# Creating the base conversation history schema per a single user
BASE_USER_CONVERSATION_HISTORY_SCHEMA = {
    'user_id': 'default_username',
    'chat_history': {}
}

BASE_USER_CONVERSATION_HISTORY_SCHEMA

{'user_id': 'default_username', 'chat_history': {}}

In [18]:
# Creating the user history from the base schema
user_history = copy.deepcopy(BASE_USER_CONVERSATION_HISTORY_SCHEMA)

In [19]:
user_history

{'user_id': 'default_username', 'chat_history': {}}

## Converting the LangChain Messages to JSON
One great thing about LangChain are the many integrations it offers for connections to many of your favorite online services. To keep things simple, I want to save my chat history as a JSON file. Now as you saw in the previous section, I'm a bit picky with my choice of schema. This means that we are going to need to figure out a way to convert the LangChain messages to JSON.

In [20]:
def lc_session_to_json_session(lc_current_session_history):
    '''
    Converts the current LangChain session history to a JSON session history

    Inputs:
        - lc_current_session_history (LangChain ChatMessageHistory): The LangChain session history

    Returns:
        - json_current_session_history (list): The LangChain session history now transformed into JSON session history
    '''
    # Instantiating a list to hold the JSON outputs
    json_current_session_history = []

    # Converting the LangChain messages using LangChain's built in json() function
    lc_generated_json = json.loads(lc_current_session_history.json())

    # Iterating over each of the LangChain messages
    for message in lc_generated_json['messages']:

        # Determining the action based on if message is Human type
        if message['type'] == 'human':

            conversation_json = {
                'role': 'user',
                'content': message['content'],
            }
            json_current_session_history.append(conversation_json)
        
        # Determining the action based on if message is AI type
        elif message['type'] == 'ai':

            conversation_json = {
                'role': 'assistant',
                'content': message['content'],
                'metadata': message['response_metadata']
            }
            json_current_session_history.append(conversation_json)
        
    return json_current_session_history

In [21]:
# Updating the user history with our demo conversation histories converted now to JSON
user_history['chat_history'][demo_id_1] = lc_session_to_json_session(lc_current_session_history = lc_user_conversation_history[demo_id_1])
user_history['chat_history'][demo_id_2] = lc_session_to_json_session(lc_current_session_history = lc_user_conversation_history[demo_id_2])
user_history

{'user_id': 'default_username',
 'chat_history': {'conv_id_4c49e021_4dc1_4b2f_9892_511e810fb7e3': [{'role': 'user',
    'content': 'What is the capital of Illinois?'},
   {'role': 'assistant',
    'content': "The capital city of Illinois is Springfield. It's located in the central part of the state and is the county seat of Sangamon County. Springfield is known for its rich history, including being the site of Abraham Lincoln's presidential library and his former home, which is now a National Historic Site.",
    'metadata': {'model_name': 'mistralai/Mistral-7B-Instruct-v0.2',
     'timestamp': '2024-05-02 14:56:24.750245+00:00',
     'like_data': None,
     'hyperparameters': {'temp': 0.7, 'max_tokens': 1000}}},
   {'role': 'user', 'content': 'What is the largest city in that state?'},
   {'role': 'assistant',
    'content': "The largest city in Illinois is Chicago. Chicago is located in the northeastern part of the state and is the third most populous city in the United States. It's 

## Hardening the Inference Pipeline
Okay, we've come a long way so far! Let's quickly recap what we've covered so far. We have built the following things:

1. **Inference pipeline**: This is the "barebones" inference pipeline that we invoke to get a response from MLX. Because there are a few little gotchas, we had to address things like manually adding the AI message metadata as part of this step.
2. **Inference pipeline with history**: The first step did NOT manage conversation history at all. In this second step, we introduced how to manage the conversation history, and we specifically did so by being able to swap back and forth between conversations using a unique session ID.
3. **(Optional) Summary title**: This part isn't necessary, but I thought it was a nice touch. This step takes the initial interaction between the user and the AI to produce a "summary title" that we can later use for our a nice display thing in our MLX Gradio chat history.
4. **User history schema**: In this section, we instantiated the expected user history schema based on the schema we manually created in the file `data/schema.json`.
5. **LangChain current session history to JSON**: This step covered how we can convert the LangChain session history into a JSON object that we can later save to / load from a local file.

We're going to put it all together now! We're going to harden our inference pipeline by creating a massive "meta" function encapsulates all our functionality here plus saves the user's chat history out every time we interact with the model!

In [22]:
def invoke_model(prompt_text, current_session_id):
    '''
    Invokes the model using MLX and saves chat history back to a local file

    Inputs:
        - prompt_text (str): The prompt text submitted by the user
        - current_session_id (str): The current session ID

    Returns
        - response (str): The response from the AI model per the input prompt
    '''
    # Referencing global variables
    global lc_user_conversation_history
    global user_history
    global chat_history_json_location
    global chat_model
    global chat_prompt_template
    global mlx_model_parameters

    # Creating the inference chain by chaining together the chat prompt, chat model, and custom function to update metadata
    inference_chain = chat_prompt_template | RunnableLambda(correct_for_no_system_models) | chat_model | RunnableLambda(update_ai_response_metadata)

    # Creating a LangChain invokable that will run the inference pipeline and also manage chat history
    inference_chain_w_history = RunnableWithMessageHistory(
        runnable = inference_chain,
        get_session_history = get_session_history,
        input_messages_key = 'input',
        history_messages_key = 'history'
    )

    # Invoking the model with the user's input prompt and current session ID
    response = inference_chain_w_history.invoke(
        input = {'input': prompt_text},
        config = {
            'configurable': {
                'session_id': current_session_id
            }
        }
    )

    # Updating the user history with our demo conversation histories converted now to JSON
    user_history['chat_history'][current_session_id] = {
        'conversation': lc_session_to_json_session(lc_current_session_history = lc_user_conversation_history[current_session_id])
    }

    # If not added already, adding the system prompt to the current session
    if 'system_prompt' not in user_history['chat_history'][current_session_id].keys():
        user_history['chat_history'][current_session_id]['system_prompt'] = mlx_model_parameters.system_prompt

    # If not added already, adding the summary title to the current session
    if 'summary_title' not in user_history['chat_history'][current_session_id].keys():
        user_history['chat_history'][current_session_id]['summary_title'] = generate_summary_title(lc_current_session_history = lc_user_conversation_history[current_session_id], chat_model = chat_model)

    # Writing the full history back to file
    with open(chat_history_json_location, 'w') as f:
        json.dump(user_history, f, indent = 4)

    return response.content

In [23]:
# Resetting the user's history from scratch
lc_user_conversation_history = {}
user_history = copy.deepcopy(BASE_USER_CONVERSATION_HISTORY_SCHEMA)

In [24]:
# Invoking the model as we did before, except now using our new meta function that also saves chat to file
invoke_model(prompt_text = 'What is the capital of Illinois?', current_session_id = demo_id_1)
invoke_model(prompt_text = 'What is the capital of California?', current_session_id = demo_id_2)
invoke_model(prompt_text = 'What is the largest city in that state?', current_session_id = demo_id_1)
invoke_model(prompt_text = 'What is the largest city in that state?', current_session_id = demo_id_2)

'The largest city in California by population is Los Angeles. Los Angeles is located in the southern part of the state and is known for its iconic Hollywood film industry, diverse cultural scene, and world-renowned attractions such as the Hollywood Walk of Fame, Griffith Observatory, and Universal Studios Hollywood. According to the latest US Census data, the population of Los Angeles County, which includes the city of Los Angeles, is over 10 million people, making it the most populous county'

## Loading the Chat History from File
Before we can call it a day, we need a means to now load this JSON chat history back from file into something that LangChain can work with.

In [25]:
def load_chat_history_from_file(chat_history_json_location):
    '''
    Loads the chat history from file

    Inputs:
        - chat_history_json_location (str): The location of where the chat history JSON file resides

    Returns:
        - user_history (dict): A dictionary representing the user history that will be saved back to file
        - lc_user_conversation_history (dict): A LangChain managed version of the user's conversation history
    '''

    # Instantiating an empty dictionary to hold the LangChain (LC) user conversation history
    lc_user_conversation_history = {}

    # Loading the user history from the local JSON file
    with open(chat_history_json_location, 'r') as f:
        user_history = json.load(f)

    # Iterating over each conversation in the user history
    for conversation_id, conversation in user_history['chat_history'].items():
        
        # Instantiating a list to keep track of the chat interaction
        chat_interaction = []

        # Instantiating a LangChain chat message history object
        lc_chat_message_history = ChatMessageHistory()

        # Iterating over each chat interaction in the conversation history
        for chat_interaction in conversation['conversation']:

            # Appending any user messages as a LangChain HumanMessage
            if chat_interaction['role'] == 'user':
                lc_chat_message_history.add_message(HumanMessage(content = chat_interaction['content']))

            # Appending any assistant messages as a LangChain AIMessage
            if chat_interaction['role'] == 'assistant':
                lc_chat_message_history.add_message(AIMessage(content = chat_interaction['content'], metadata = chat_interaction['metadata']))

        # Appending the full conversation and metadata to the LC user conversation history with conversation ID as the key
        lc_user_conversation_history[conversation_id] = lc_chat_message_history

    return user_history, lc_user_conversation_history

In [26]:
# Loading in user history from file
user_history, lc_user_conversation_history = load_chat_history_from_file(chat_history_json_location = chat_history_json_location)

## A Real Life Use Case
Okay, we now have the essential basic framework to start using our chatbot and saving our chat history to a local JSON file. We will jump back and forth between different use cases to ensure that what we are building is resilient to everyday use. Let's first begin by starting from scratch.

In [27]:
# Resetting the user's history from scratch
lc_user_conversation_history = {}
user_history = copy.deepcopy(BASE_USER_CONVERSATION_HISTORY_SCHEMA)

### Simulating a First Conversation

In [28]:
# Creating demo ID 3 (Just to not confuse with varibles used previously in the notebook)
demo_id_3 = 'conv_id_' + str.replace(str(uuid.uuid4()), '-', '_')

In [29]:
# Invoking the model to get a response for demo ID 3
invoke_model(prompt_text = 'What is the capital of Illinois?', current_session_id = demo_id_3)

"The capital city of Illinois is Springfield. It's located in the central part of the state and is the county seat of Sangamon County. Springfield is known for its rich history, including being the site of Abraham Lincoln's presidential library and his former home, which is now a National Historic Site."

### Starting a New (Second) Conversation

In [30]:
# Creating demo ID 4 (Just to not confuse with varibles used previously in the notebook)
demo_id_4 = 'conv_id_' + str.replace(str(uuid.uuid4()), '-', '_')

In [31]:
# Invoking the model to get a response for demo ID 4
invoke_model(prompt_text = 'What is the capital of California?', current_session_id = demo_id_4)

"The capital city of California is Sacramento. Sacramento is located in the central part of the state and is known for its rich history, cultural diversity, and various attractions such as Old Sacramento, the California State Capitol, and the California State Railroad Museum. It's also the seat of government for California and is home to various state government offices and agencies."

### Jumping Back to First Conversation

In [32]:
# Invoking the model to get a response for demo ID 3
invoke_model(prompt_text = 'What is the largest city in that state?', current_session_id = demo_id_3)

"The largest city in Illinois is Chicago. Chicago is located in the northeastern part of the state and is the third most populous city in the United States. It's known for its iconic skyline, major industries, cultural institutions, and its role as a global hub for commerce, transportation, and technology. Chicago is also famous for its architecture, museums, and its food scene, particularly its deep-dish pizza and Chicago-style hot dogs."

### Loading It All Back In from File

In [33]:
# Loading in user history from file
user_history, lc_user_conversation_history = load_chat_history_from_file(chat_history_json_location = chat_history_json_location)

### Starting a Third (Final) Conversation

In [34]:
# Creating demo ID 5 (Just to not confuse with varibles used previously in the notebook)
demo_id_5 = 'conv_id_' + str.replace(str(uuid.uuid4()), '-', '_')

In [35]:
# Invoking the model to get a response for demo ID 5
invoke_model(prompt_text = 'What is the capital of New York?', current_session_id = demo_id_5)

'New York does not have a capital city because it is a state in the United States. The capital city of New York State is Albany. However, New York City is the most populous city in the state and the largest city in the United States, and it is not the capital city. Albany is located about 150 miles northwest of New York City.'

### Returning to the First Conversation

In [36]:
# Invoking the model to get a response for demo ID 3
invoke_model(prompt_text = 'What are some fun things to do in that city?', current_session_id = demo_id_3)

"Chicago offers a wide range of activities for visitors of all ages and interests. Here are some popular things to do in Chicago:\n\n1. Explore the city's iconic architecture: Take a river cruise or architectural boat tour to learn about the city's famous buildings and their architects.\n2. Visit museums: Chicago is home to world-class museums such as the Art Institute of Chicago, the Museum of Science and Industry, and the Shedd Aquarium"

### Loading it all in from file one last time!

In [37]:
# Resetting the user's history from scratch (to ensure everything loads from file correctly)
lc_user_conversation_history = {}
user_history = copy.deepcopy(BASE_USER_CONVERSATION_HISTORY_SCHEMA)

In [38]:
# Loading in user history from file
user_history, lc_user_conversation_history = load_chat_history_from_file(chat_history_json_location = chat_history_json_location)
lc_user_conversation_history

{'conv_id_44642eb7_3066_4cd4_8d76_1e0279478cea': ChatMessageHistory(messages=[HumanMessage(content='What is the capital of Illinois?'), AIMessage(content="The capital city of Illinois is Springfield. It's located in the central part of the state and is the county seat of Sangamon County. Springfield is known for its rich history, including being the site of Abraham Lincoln's presidential library and his former home, which is now a National Historic Site.", metadata={}), HumanMessage(content='What is the largest city in that state?'), AIMessage(content="The largest city in Illinois is Chicago. Chicago is located in the northeastern part of the state and is the third most populous city in the United States. It's known for its iconic skyline, major industries, cultural institutions, and its role as a global hub for commerce, transportation, and technology. Chicago is also famous for its architecture, museums, and its food scene, particularly its deep-dish pizza and Chicago-style hot dogs.